In [2]:
import numpy as np
import tensorflow as tf
import os
import time

In [ ]:
class FPNN:
    def __init__(self, sigma=8):
        self.sigma = sigma
        
    def train(self, train_data, train_labels, batch_size=256):
        train_data = tf.convert_to_tensor(train_data)
        train_labels = tf.convert_to_tensor(train_labels)
    
        # how do I load batches pls
        train_image_batch, train_label_batch = tf.train.shuffle_batch([train_data, train_labels],batch_size=BATCH_SIZE)
        
        
        ## HARD THINGS ##
        # field probing https://github.com/yangyanli/caffe/blob/field_probing/src/caffe/layers/field_probing_layer.cpp
        # rotation z? https://github.com/yangyanli/caffe/blob/field_probing/src/caffe/layers/transform_3d_layer.cpp
        # gaussian https://github.com/yangyanli/caffe/blob/field_probing/src/caffe/layers/gaussian_layer.cpp
        # dot product https://github.com/yangyanli/caffe/blob/field_probing/src/caffe/layers/dot_product_layer.cpp
        
        ## EASY THINGS ##
        # batch norm
        # relu
        # fc0
        # batch norm
        # relu
        # dropout
        # fc1
        # batch norm
        # relu
        # dropout
        # fc2
        # batch norm
        # relu
        # dropout
        # fc3
        # softmax and accuracy
        ## BELOW ##
        
        bn_dp = tf.contrib.layers.batch_norm(dp, center=True, scale=True, is_training = True, name='bn_dp')
        relu_dp = tf.nn.relu(bn_dp, name='relu_dp')
        
        fc0 = fc_layer(relu_dp, 'fc0', 1024)
        bn0 = tf.contrib.layers.batch_norm(fc0, center=True, scale=True, is_training = True, name='bn_0')
        relu0 = tf.nn.relu(bn0, name='relu0')
        dropout0 = tf.nn.dropout(relu0, keep_prob = 0.5, name='dropout0')
        fc1 = fc_layer(dropout0, 'fc1', 1024)
        bn1 = tf.contrib.layers.batch_norm(fc1, center=True, scale=True, is_training = True, name='bn_1')
        relu1 = tf.nn.relu(bn1, name='relu1')
        dropout1 = tf.nn.dropout(relu1, keep_prob = 0.5, name='dropout1')
        fc2 = fc_layer(dropout1, 'fc2', 1024)
        bn2 = tf.contrib.layers.batch_norm(fc2, center=True, scale=True, is_training = True, name='bn_2')
        relu2 = tf.nn.relu(bn2, name='relu2')
        dropout2 = tf.nn.dropout(relu2, keep_prob = 0.5, name='dropout2')
        fc3 = fc_layer(dropout2, 'fc3', 40, 0)
        
        y_hat = tf.nn.softmax(fc3, name='prob')
    
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=fc3, labels=train_label_batch))
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_hat, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

        # run tf session and save
        
    def fc_layer(bottom, name, num, intial_bias=1):
        with tf.variable_scope(name):
            shape = bottom.get_shape().as_list()
            dim = 1
            for d in shape[1:]:
                dim *= d
            x = tf.reshape(bottom, [-1, dim])

            weights = tf.Variable(tf.truncated_normal([dim, num], stddev=0.05), name='weights')
            if initial_bias:
                biases = tf.Variable(tf.ones([num]), name='biases')
            else:
                biases = tf.Variable(tf.zeros([num]), name='biases')
            fc = tf.nn.bias_add(tf.matmul(x, weights), biases)

            return fc    
    
    
    
    def test(self, test_data):